# HOW TO ADD A NEW CLASS TO OBJECT DETECTION PIPELINE?

In [ ]:
## Uncomment command below to kill current job:
#!neuro kill $(hostname)

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
from detection.model import get_model
from detection.coco_subset import CLS_SELECT, COLORS, N_COCO_CLASSES
from detection.dataset import get_transform
from detection.visualisation import show_legend, predict_and_show
from detection.train import train
import torch
from pathlib import Path
from random import choice
from PIL import Image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

## Downloading the dataset

In [ ]:
! [ ! -f ../data/coco-retail.zip ] && wget http://data.neu.ro/coco-retail.zip -O ../data/coco-retail.zip

In [ ]:
! [ ! -d ../data/coco ] && unzip -q ../data/coco-retail.zip -d ../data

## Dataset overview

We took **25 classes** from COCO-dataset which can be seen on the shelves as retail products.
Since our goal is show how new category can be added to detection pipeline 
(without long training process), 
we work with only 100 photos and don't make any train/val split. 
Of course, with these settings, the model will be prone to over-fitting, 
but training on the whole dataset will take too much time.

In [ ]:
data_dir = Path('../data/coco/mini_coco/')

In [ ]:
show_legend(list(CLS_SELECT.keys()), COLORS)

## Evaluation of model, trained on 24 classes

Let's load our model, trained on 24 classes (without class #25 - **sport ball**).

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:
path_to_ckpt = '../data/coco/weights/24_classes.ckpt'

model = get_model(n_classes=N_COCO_CLASSES - 1)
model.load_state_dict(torch.load(path_to_ckpt, map_location=device))
model.eval();

Now let's make sure that the model can't recognize the 25'th class.

In [ ]:
def get_im_with_extra_class():

    images_with_extra_class = [
        '000000091595.jpg', '000000331474.jpg', '000000371042.jpg',
        '000000209027.jpg', '000000032611.jpg', '000000002139.jpg',
        '000000050407.jpg', '000000345466.jpg', '000000465530.jpg'
    ]
    
    im_path = data_dir / 'train' / 'images' / choice(images_with_extra_class)
    
    im_pil = Image.open(im_path)
    im_tensor, _ = get_transform(False)(im_pil, None)
    
    print(im_path.name)
    return im_pil, im_tensor

In [ ]:
im_pil, im_tensor = get_im_with_extra_class()
# Note: on CPU, inference of an image takes around 1 minute
predict_and_show(model.to(device), im_pil, im_tensor.to(device))

## Add 25'th class

The simplest way to add new classes includes 2 steps:
* Increasing the number of output logits; 
* Training / fine-tuning process.

In [ ]:
# Now we can train our model or load it from prepared checkpoint

want_finetune = False

if want_finetune:
    # fine-tune previous model
    model_ext = get_model(n_classes=N_COCO_CLASSES - 1).to(device)
    model_ext.load_state_dict(torch.load(path_to_ckpt, map_location=device))
    n_features = model_ext.roi_heads.box_predictor.cls_score.in_features
    model_ext.roi_heads.box_predictor = FastRCNNPredictor(n_features, N_COCO_CLASSES)
    
    train(model=model_ext, data_dir=data_dir, prev_ckpt=None, n_epoch=50,
          batch_size=2, n_workers=4, ignore_labels=(), need_save=False)

else:
    # load from ckpt
    path_to_ckpt_ext = '../data/coco/weights/25_classes.ckpt'
    model_ext = get_model(n_classes=N_COCO_CLASSES)
    model_ext.load_state_dict(torch.load(path_to_ckpt_ext, map_location=device))
    

Let's check that our model can now recognize the added 25'th class.

In [ ]:
im_pil, im_tensor = get_im_with_extra_class()

predict_and_show(model_ext.to(device), im_pil, im_tensor.to(device))